In [ ]:
import os
sys.path.append(os.path.abspath(".."))
from shopping_agent import run_shopping_agent

: 

In [ ]:
instruction = "لطفاً دراور چهار کشو (کد D14) را برای من تهیه کنید."
# instruction = "عرض پارچه تریکو جودون 1/30 لاکرا گردباف نوریس به رنگ زرد طلایی چقدر است؟"
# instruction = "کمترین قیمت در این پایه برای گیاه طبیعی بلک گلد بنسای نارگل کد ۰۱۰۸ چقدر است؟"
# instruction = "یک لپ تاپ مناسب بهم پیشنهاد کن."
# instruction = """
# چند تا کالا پیدا کن که برای فروشگاه های متفاوتی هستند و بهم معرفی کن.
# """
instruction = """
random_key رو برای اونی که مناسبتره بنویس.
"کدام یک از این ماگ‌های خرید ماگ-لیوان هندوانه فانتزی و کارتونی کد 1375 یا ماگ لته خوری سرامیکی با زیره کد 741 دارای سبک کارتونی و فانتزی بوده و برای کودکان یا نوجوانان مناسب‌تر است؟"
"""
# instruction = """
# What is the average price of all products in Torob?
# """
result = await run_shopping_agent(instruction)

: 

In [ ]:
result

: 

In [ ]:
dict(result.output)

: 

In [ ]:
result.usage()

: 

In [ ]:
result.all_messages()

: 

: 